In [1]:
import os
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf


spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
from pyspark.sql.functions import col

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [2]:
# download a Postgres driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-04 16:13:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.4’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-08-04 16:13:52 (6.50 MB/s) - ‘postgresql-42.2.16.jar.4’ saved [1002883/1002883]



In [3]:
# start a Spark session with driver
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Olympic-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://olympicclassproject.s3.us-east-2.amazonaws.com/cleandatabmi.xlsb.csv"
spark.sparkContext.addFile(url)
cleandatadf = spark.read.csv(SparkFiles.get("cleandatabmi.xlsb.csv"), sep=",", header=True, inferSchema=True)

In [5]:
# Show DataFrame
cleandatadf.show()

+--------------------+---+---+------+------+-----------+-----------+------+--------------------+------+---+
|                Name|Sex|Age|Height|Weight|       Team|      Games|Season|               Event| Medal|BMI|
+--------------------+---+---+------+------+-----------+-----------+------+--------------------+------+---+
|Juhamatti Tapio A...|  M| 28|   184|    85|    Finland|2014 Winter|Winter|Ice Hockey Men's ...|Bronze| 25|
|Paavo Johannes Aa...|  M| 28|   175|    64|    Finland|1948 Summer|Summer|Gymnastics Men's ...|Bronze| 21|
|Paavo Johannes Aa...|  M| 28|   175|    64|    Finland|1948 Summer|Summer|Gymnastics Men's ...|  Gold| 21|
|Paavo Johannes Aa...|  M| 28|   175|    64|    Finland|1948 Summer|Summer|Gymnastics Men's ...|  Gold| 21|
|Paavo Johannes Aa...|  M| 28|   175|    64|    Finland|1948 Summer|Summer|Gymnastics Men's ...|  Gold| 21|
|Paavo Johannes Aa...|  M| 32|   175|    64|    Finland|1952 Summer|Summer|Gymnastics Men's ...|Bronze| 21|
|  Kjetil Andr Aamodt|  M| 2

In [6]:
#Assign Medal types a number
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Medal", outputCol="Medals") 
cleandatadf = indexer.fit(cleandatadf).transform(cleandatadf) 
cleandatadf = cleandatadf.drop('Medal')
cleandatadf.show()

+--------------------+---+---+------+------+-----------+-----------+------+--------------------+---+------+
|                Name|Sex|Age|Height|Weight|       Team|      Games|Season|               Event|BMI|Medals|
+--------------------+---+---+------+------+-----------+-----------+------+--------------------+---+------+
|Juhamatti Tapio A...|  M| 28|   184|    85|    Finland|2014 Winter|Winter|Ice Hockey Men's ...| 25|   0.0|
|Paavo Johannes Aa...|  M| 28|   175|    64|    Finland|1948 Summer|Summer|Gymnastics Men's ...| 21|   0.0|
|Paavo Johannes Aa...|  M| 28|   175|    64|    Finland|1948 Summer|Summer|Gymnastics Men's ...| 21|   1.0|
|Paavo Johannes Aa...|  M| 28|   175|    64|    Finland|1948 Summer|Summer|Gymnastics Men's ...| 21|   1.0|
|Paavo Johannes Aa...|  M| 28|   175|    64|    Finland|1948 Summer|Summer|Gymnastics Men's ...| 21|   1.0|
|Paavo Johannes Aa...|  M| 32|   175|    64|    Finland|1952 Summer|Summer|Gymnastics Men's ...| 21|   0.0|
|  Kjetil Andr Aamodt|  M| 2

In [7]:
pandas_df=cleandatadf.toPandas()

In [8]:
#Drop non-feature columns
pandas_df.drop(['Name', "Sex", "Team", "Games", "Season", "Event"], inplace=True, axis=1)
pandas_df.head()

,Age,Height,Weight,BMI,Medals
0,28,184,85,25,0.0
1,28,175,64,21,0.0
2,28,175,64,21,1.0
3,28,175,64,21,1.0
4,28,175,64,21,1.0


In [9]:
pandas_df.nunique(axis=0)

Age        52
Height    111
Weight    160
BMI        40
Medals     80
dtype: int64

In [10]:
#create categorical variable list for encoding
olympic_cat = pandas_df.dtypes[pandas_df.dtypes == "object"].index.tolist()
olympic_cat

['Age', 'BMI']

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(pandas_df[olympic_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(olympic_cat)


In [12]:
#Merge encoded values and numbered medals and drop original columns
pandas_df = pandas_df.merge(encode_df,left_index=True, right_index=True)
pandas_df = pandas_df.drop(olympic_cat,1)

pandas_df.head()

,Height,Weight,Medals,Age_13,Age_14,Age_15,Age_16,Age_17,Age_18,Age_19,Age_20,Age_21,Age_22,Age_23,Age_24,Age_25,Age_26,Age_27,Age_28,Age_29,Age_30,Age_31,Age_32,Age_33,Age_34,Age_35,Age_36,Age_37,Age_38,Age_39,Age_40,Age_41,Age_42,Age_43,Age_44,Age_45,Age_46,Age_47,Age_48,Age_49,...,BMI_13,BMI_14,BMI_15,BMI_16,BMI_17,BMI_18,BMI_19,BMI_20,BMI_21,BMI_22,BMI_23,BMI_24,BMI_25,BMI_26,BMI_27,BMI_28,BMI_29,BMI_30,BMI_31,BMI_32,BMI_33,BMI_34,BMI_35,BMI_36,BMI_37,BMI_38,BMI_39,BMI_40,BMI_41,BMI_42,BMI_43,BMI_44,BMI_45,BMI_46,BMI_47,BMI_51,BMI_56,BMI_Bronze,BMI_Gold,BMI_Silver
0,184,85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,175,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,175,64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,175,64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,175,64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Split preprocessed data into features and target arrays
y = pandas_df["Medals"].values
X = pandas_df.drop(["Medals"],1).values

# Split the preprocessed data into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [14]:
# Create StandardScaler instance
scaler = StandardScaler()

# Fit StandardScaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Define model
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  60
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                5700      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 244       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 5,969
Trainable params: 5,969
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
708/708 [==============================] - 2s 2ms/step - loss: -0.0774 - accuracy: 0.3341
Epoch 2/100
708/708 [==============================] - 1s 2ms/step - loss: -0.4503 - accuracy: 0.3340
Epoch 3/100
708/708 [==============================] - 1s 2ms/step - loss: -0.6327 - accuracy: 0.3339
Epoch 4/100
708/708 [==============================] - 1s 2ms/step - loss: -0.7977 - accuracy: 0.3341
Epoch 5/100
708/708 [==============================] - 1s 2ms/step - loss: -0.9594 - accuracy: 0.3336
Epoch 6/100
708/708 [==============================] - 1s 2ms/step - loss: -1.1194 - accuracy: 0.3340
Epoch 7/100
708/708 [==============================] - 1s 2ms/step - loss: -1.2810 - accuracy: 0.3340
Epoch 8/100
708/708 [==============================] - 1s 2ms/step - loss: -1.4621 - accuracy: 0.3345
Epoch 9/100
708/708 [==============================] - 1s 2ms/step - loss: -1.6377 - accuracy: 0.3342
Epoch 10/100
708/708 [==============================] - 1s 2ms/step - loss: -1.824

In [18]:
# Evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

236/236 - 0s - loss: -1.0933e+01 - accuracy: 0.3343
Loss: -10.933160781860352, Accuracy: 0.3343493342399597
